In [1]:
import numpy as np
import tensorflow as tf

# **Loaded IMDB dataset**

---



---



In [2]:
(X_train,Y_train),(X_test,Y_test)=tf.keras.datasets.imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 1s 0us/step


# **Padded sequences to make all vector size equal**

---



---



In [3]:
X_train_padding=tf.keras.preprocessing.sequence.pad_sequences(X_train,maxlen=200,padding='post')
X_test_padding=tf.keras.preprocessing.sequence.pad_sequences(X_test,maxlen=200,padding='post')

# **Defined Positional encoding function**

---



---



In [5]:
def positional_encoding(seq_len,d_model):
  # seq_len,d_model=seq_len,d_model
  pos_encoding=np.zeros((seq_len,d_model))
  for i in range(seq_len):
    for j in range(0,d_model,2):
      pos_encoding[i,j]=np.sin(i/(10000**(j/d_model)))
      pos_encoding[i,j+1]=np.cos(i/(10000**((j+1)/d_model)))
  return tf.cast(pos_encoding,dtype=tf.float32)

# **Built Custom Transformer model**

---



---




In [6]:
class transformer(tf.keras.layers.Layer):
  def __init__(self,d_model,num_heads,dff,rate=0.1):
    super(transformer,self).__init__()
    self.mha=tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=d_model)
    self.ffn=tf.keras.Sequential([
        tf.keras.layers.Dense(dff,activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])
    self.norm1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.dropout=tf.keras.layers.Dropout(rate=rate)
    self.dropout2=tf.keras.layers.Dropout(rate=rate)

  def call(self,X,training):
    attn_output=self.mha(X,X)
    attn_output=self.dropout(attn_output,training=training)
    out1=self.norm1(X+attn_output)
    fnn_output=self.ffn(out1)
    fnn_output=self.dropout2(fnn_output,training=training)
    out2=self.norm2(out1+fnn_output)
    return out2

# **Defined Model by layering**

---



---



In [7]:
seq_len=200
d_model=128
num_heads=4
dff=512
vocab_size=10000

inputs = tf.keras.Input(shape=(seq_len,))
x = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
pos_encoding = positional_encoding(seq_len, d_model)
x=x+pos_encoding
x = transformer(d_model, num_heads, dff)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# **Trained model on IMDB dataset**

---



---



In [8]:
model.fit(X_train_padding,Y_train,epochs=5,validation_data=(X_test_padding,Y_test))

Epoch 1/5
782/782 [==============================] - 57s 58ms/step - loss: 0.4710 - accuracy: 0.7386 - val_loss: 0.3273 - val_accuracy: 0.8606
Epoch 2/5
782/782 [==============================] - 25s 32ms/step - loss: 0.2390 - accuracy: 0.9034 - val_loss: 0.2999 - val_accuracy: 0.8749
Epoch 3/5
782/782 [==============================] - 22s 28ms/step - loss: 0.1687 - accuracy: 0.9376 - val_loss: 0.4076 - val_accuracy: 0.8457
Epoch 4/5
782/782 [==============================] - 25s 32ms/step - loss: 0.1149 - accuracy: 0.9588 - val_loss: 0.4444 - val_accuracy: 0.8524
Epoch 5/5
782/782 [==============================] - 20s 26ms/step - loss: 0.0736 - accuracy: 0.9744 - val_loss: 0.5078 - val_accuracy: 0.8535
